In [1]:
import sklearn.metrics
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid, PPI
from torch_geometric.nn import GAT
from datetime import datetime

In [3]:
for dataset in ['citeseer', 'cora', 'pubmed']:
  start = datetime.now()
  dataset = Planetoid(root=f'../data/{dataset}', name=dataset)
  # Define model and optimizer
  model = GAT(
      in_channels=dataset.num_features,
      out_channels=dataset.num_classes,
      hidden_channels=8,
      num_layers=2,
      heads=8,
      dropout=0.6,
      act='elu',
      act_first=True
  )
  # {'PairNorm', 'GraphSizeNorm', 'HeteroLayerNorm', 'InstanceNorm', 'BatchNorm', 'DiffGroupNorm', 'GraphNorm', 'HeteroBatchNorm', 'MessageNorm', 'MeanSubtractionNorm', 'LayerNorm'}
  optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

  # Train model
  for epoch in range(200):
      model.train()
      optimizer.zero_grad()
      out = model(dataset.x, dataset.edge_index)
      #loss = F.nll_loss(out[dataset.train_mask], dataset.y[dataset.train_mask])
      loss = F.cross_entropy(out[dataset.train_mask], dataset.y[dataset.train_mask])
      loss.backward()
      optimizer.step()

      # Evaluate model
      model.eval()
      pred = model(dataset.x, dataset.edge_index).argmax(dim=1)
      correct = int(pred[dataset.train_mask].eq(dataset.y[dataset.train_mask]).sum().item())
      acc = correct / int(dataset.train_mask.sum())
      # print(f'Epoch {epoch + 1:03d}, Loss: {loss:.4f}, Test Acc: {acc:.4f}')


  # Test the model
  model.eval()
  out = model(dataset.x, dataset.edge_index)
  pred = out.argmax(dim=1)
  acc = pred[dataset.test_mask].eq(dataset.y[dataset.test_mask]).sum().item() / int(dataset.test_mask.sum())
  print('\n\n*****************************************************************************************************\n')
  print(f'                                         {dataset} ')
  print(f'                                         Total Epochs: 200')
  print(f'                                         Test Accuracy: {acc:.4f}')
  print(f'                                         Time Taken: {datetime.now() - start}')
  print('\n*****************************************************************************************************\n\n')

Processing...
Done!




*****************************************************************************************************

                                         citeseer() 
                                         Total Epochs: 200
                                         Test Accuracy: 0.6670
                                         Time Taken: 0:00:29.233131

*****************************************************************************************************




Processing...
Done!




*****************************************************************************************************

                                         cora() 
                                         Total Epochs: 200
                                         Test Accuracy: 0.8070
                                         Time Taken: 0:00:18.929057

*****************************************************************************************************




Processing...
Done!




*****************************************************************************************************

                                         pubmed() 
                                         Total Epochs: 200
                                         Test Accuracy: 0.7770
                                         Time Taken: 0:01:14.338753

*****************************************************************************************************




In [5]:

ppi_train = PPI('../data/ppi/')


Extracting ../data/ppi/ppi.zip
Processing...
Done!


In [6]:
model = GAT(
    in_channels=ppi_train.num_features,
    out_channels=ppi_train.num_classes,
    hidden_channels=8,
    num_layers=2,
    heads=8,
    dropout=0.6,
    act='elu',
    act_first=True
)


In [7]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


In [ ]:
start = datetime.now()
# Train model
for epoch in range(2):
  print(epoch)
  model.train()
  optimizer.zero_grad()
  out = model(ppi_train.x, ppi_train.edge_index)
  loss = F.cross_entropy(out, ppi_train.y)
  loss.backward()
  optimizer.step()

  # Evaluate model
  model.eval()
  pred = model(ppi_train.x, ppi_train.edge_index) > .5
  f1 = sklearn.metrics.f1_score(ppi_train.y.detach().numpy(), pred.detach().numpy(), average='micro')
  #print(f'Epoch {epoch + 1:03d}, Loss: {loss:.4f}, F1: {f1}')


# Test the model
ppi_test = PPI('../data/ppi/', 'test')
model.eval()
out = model(ppi_test.x, ppi_test.edge_index) > .5
f1 = sklearn.metrics.f1_score(ppi_test.y.detach().numpy(), out.detach().numpy(), average='micro')
print('\n\n*****************************************************************************************************\n')
print(f'                                         PPI Dataset ')
print(f'                                         Total Epochs: 200')
print(f'                                         F1 Score: {f1:.4f}')
print(f'                                         Time Taken: {datetime.now() - start}')
print('\n*****************************************************************************************************\n\n')

0
